In [27]:
# [1]包导入
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import missingno as msno
import warnings
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [28]:
# [2]全局变量
TARGET='SalePrice'
oTARGET='SalePrice'
# 设置支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体支持中文
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号
# 屏蔽此类警告
import warnings
warnings.filterwarnings("ignore", message="Could not find the number of physical cores")
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # 或其他你机器的逻辑核心数

In [29]:
# [3]数据集读取
otrain_data = pd.read_csv('train.csv')
otest_data = pd.read_csv('test.csv')
train_data = otrain_data.copy()
test_data = otest_data.copy()         # 原始数据备份
train_data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [30]:
# [5]筛选训练集和测试集都存在的列
cols = [TARGET]         # target列一定保留
for c in train_data.columns:
    if c in test_data.columns:
        cols.append(c)
train_data = train_data[cols]
test_data = test_data[cols[1:]]        # test_data不包含结果列

In [31]:
X_train = train_data.drop(TARGET, axis=1)
y_train = train_data[TARGET]
X_test = test_data

X_train_split, X_valid, y_train_split, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)


In [32]:
# 1. 自动识别所有非数值列（object 或 category 类型）
cat_cols = X_train_split.select_dtypes(include=['object', 'category']).columns.tolist()

# 2. 对训练集、验证集、测试集进行独热编码
X_train_encoded = pd.get_dummies(X_train_split, columns=cat_cols, drop_first=True)
X_valid_encoded = pd.get_dummies(X_valid, columns=cat_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# 3. 对齐验证集和测试集的列，使其与训练集完全一致
X_valid_encoded = X_valid_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# 4. 建模
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_encoded, y_train_split)

# 5. 验证集预测与评估
y_pred = model.predict(X_valid_encoded)
log_mse = mean_squared_error(np.log1p(y_valid), np.log1p(y_pred))
log_rmse = np.sqrt(log_mse)
print(f"验证集上的对数 RMSE：{log_rmse:.4f}")

# 6. 测试集预测（如需要）
# y_test_pred = model.predict(X_test_encoded)


验证集上的对数 RMSE：0.1442


In [35]:
# 预测
preds = model.predict(X_test_encoded)

# 6. 输出结果（如果你打算提交）
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': preds
})
submission.to_csv('submission.csv', index=False)